In [54]:
import torch
from torch import nn
import torch.optim as optim
import scipy.io as spio

## Define DNN Model

In [ ]:
class DNN(nn.Module, config=None):
    
    def __init__(self):
        super().__init__()
        self.learning_rate=0.005
        self.batch_size=128
        self.n_band = 40
        self.n_hidden1 = 512
        self.n_hidden2 = 32
        self.n_classes = 2
        self.EPOCHES=18000
        self.input_layer = nn.Linear(n_band,n_hidden1)
        self.hidden_layer = nn.Linear(n_hidden1,n_hidden2)
        self.output_layer = nn.Linear(n_hidden2,n_classes)
        self.activation_fun = nn.ReLU(inplace=False)
        self.criterion = nn.CrossEntropyLoss(reduction='mean') 
        
    def forward(self,x):
        x = self.input_layer(x)
        x = self.activation_fun(x)
        x = self.hidden_layer(x)
        x = self.activation_fun(x)
        x = self.output_layer(x)
        return x
        
    

## change 1D to 2D

In [ ]:
def datachange(input):
    out = []
    for i in range(len(input)):
        if input[i] == 0:
            out.append([1, 0])
        else:
            out.append([0, 1])
    return out

In [59]:
def Train():
    # load mat file
    x, y = load_data()
    print("type of x is {}, type of y is {}".format(type(x), type(y)))
    
    

In [60]:
def load_data():
    train_all = spio.loadmat('../train_data/8+2band(25ms)/train_1106a_sharp_12.mat')
    x_train = train_all['x_data']

    train_label = spio.loadmat('../train_label/8+2band(25ms)/label_1106a_sharp_12.mat')
    y_train = train_label['y_data']
    x_train_tensors = torch.from_numpy(x_train)
    y_label_tensors = torch.from_numpy(y_train[0])
    return x_train_tensors, y_label_tensors

In [61]:
if __name__ == "__main__":
    Train()

type of x is <class 'torch.Tensor'>, type of y is <class 'torch.Tensor'>


## Reference Codes

In [ ]:
"""
def trainProcessEntry():

    train_all = spio.loadmat('train_data/8+2band(25ms)/train_1106a_sharp_12.mat')
    x_train = train_all['x_data']
    train_label = spio.loadmat('train_label/8+2band(25ms)/label_1106a_sharp_12.mat')
    y_train = train_label['y_data']
    y_train = y_train[0]

    y_train = datachange(y_train)
    # Parameters
    learning_rate = 0.01
    batch_size = 128
    n_band = 40
    n_hidden1 = 512
    n_hidden2 = 32
    n_classes = 2
    EPOCHES = 16001

    # initial the tf Graph input
    x = tf.placeholder(tf.float32, [None, n_band])
    #initial the tf Graph output
    y = tf.placeholder(tf.float32, [None, n_classes])

    # initial the weights
    weights = {
        'hidden1': tf.Variable(tf.random_normal([n_band, n_hidden1], dtype=tf.float32, stddev=0.1)),
        'hidden2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2], dtype=tf.float32, stddev=0.1)),
        'out': tf.Variable(tf.random_normal([n_hidden2, n_classes], dtype=tf.float32, stddev=0.1))
    }
    biases = {
        'hidden1': tf.Variable(tf.zeros([n_hidden1], dtype=tf.float32)),
        'hidden2': tf.Variable(tf.zeros([n_hidden2], dtype=tf.float32)),
        'out': tf.Variable(tf.zeros([n_classes], dtype=tf.float32))
    }

    # define training computation procedure
    x1 = tf.nn.relu(tf.add(tf.matmul(x, weights['hidden1']),biases['hidden1']))
    x2 = tf.nn.relu(tf.add(tf.matmul(x1, weights['hidden2']),biases['hidden2']))
    pred = tf.add(tf.matmul(x2,weights['out']),biases['out'])
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    #initial used data structures
    error_train = []
    error_test = []
    weight1 = []
    weight2 = []
    weight3 = []
    bias1 = []
    bias2 = []
    bias3 = []
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(init)



        for epoch in range(EPOCHES):
            for step in range(100):
                x_n, y_n = next_batch(batch_size, x_train, y_train)
                sess.run(train_step, feed_dict={x: x_n, y: y_n})
                acc = sess.run(accuracy, feed_dict={x: x_train, y: y_train})
                print("epoch", epoch)
                print("train : ", acc)
                error_train.append(acc)

        weight1 = weights['hidden1'].eval(sess)
        weight2 = weights['hidden2'].eval(sess)
        weight3 = weights['out'].eval(sess)
        bias1 = biases['hidden1'].eval(sess)
        bias2 = biases['hidden2'].eval(sess)
        bias3 = biases['out'].eval(sess)
        saver.save(sess, "vad_512_model")
        print("Training Finished........")
        print("The lenth of wight1 is {}".format(len(weight1)))
        print("The lenth of wight2 is {}".format(len(weight2)))
        print("The lenth of wight1 is {}".format(len(weight3)))

        print("Writing out parameters to w_20200106_h1_512_ep20000")
        spio.savemat('weight/8+2band(25ms)/w_20200106_h1_512_ep20000.mat',
                     {'w1': weight1, 'w2': weight2, 'w3': weight3, 'b1': bias1, 'b2': bias2, 'b3': bias3})

    plt.plot(range(len(error_train)), error_train, 'b', label='Training accuracy')
    plt.title('accuracy')
    plt.xlabel('epoch', fontsize=16)
    plt.ylabel('accuracy', fontsize=16)
    plt.legend()
    plt.figure()
    plt.show()
    
   ####################################################################
   
   model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))
# Define the loss
criterion = nn.NLLLoss()
# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.SGD(model.parameters(), lr=0.003)
epochs = 5
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        # Flatten MNIST images into a 784 long vector
        images = images.view(images.shape[0], -1)
    
        # Training pass
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")
    
"""